# Import

In [27]:
import numpy as np


# バックプロパゲーション

## 下準備
<img src="../images/関数と変数の関係性.png" width="400">


In [28]:
# DeZeroで使用する変数クラス
class Variable:
    def __init__(self, data):
        self.data = data  #　通常の値
        self.grad=None    #　dataに対応した微分値
        self.creator=None #このクラスの変数が生成される関数　例：y=f(x)のとき　yにとってfがcreator

    # creatorのsetter
    def set_creator(self,func):
        self.creator=func

    # 逆伝播を行う関数
    def backward(self):
        f=self.creator #関数を取得
        if f is not None:
            x=f.input # 2.関数の入力を所得
            x.grad=f.backward(self.grad) # 3. 関数のbackwardメソッドを呼ぶ
            x.backward() #　自分より1つ前の変数のbackwardメソッドを呼ぶ（再起）

# Variableクラスを処理する関数を定義するクラス
# このクラスを基底クラスとして、共通する機能を実現
class Function:
    def __call__(self, input):
        x = input.data  #　データを取り出す
        y = self.forward(x) #　実際の計算
        output = Variable(y) # Variableとして返す
        output.set_creator(self) # 出力変数に生みの親を覚えさせる
        self.input=input #　入力された変数を覚える
        self.output=output #　出力も覚えさせる
        return output
    #順伝播を行う機能
    def forward(self,x):
        raise NotImplementedError() #意図的に例外を発生させる
    # 微分の罫線を行う逆伝播の機能
    def backward(self,gy):
        raise NotImplementedError()

# function classを継承して利便性向上
class Square(Function):
    def forward(self, x):
        return x**2
    #y=x^2の微分を計算dy/dx=2x
    def backward(self,gy):
        x=self.input.data
        gx=2*x*gy
        return gx

class Exp(Function):
    def forward(self, x):
        y=np.exp(x)
        return y
    def backward(self,gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx
        

## 逆伝播
<img src="../images/計算グラフの例.png" width="800">

In [29]:
#　順伝播
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)
print(y.data)
# 逆伝播（再起的に行わない場合）
y.grad=np.array(1.0)
C=y.creator # 1. 関数を取得
b=C.input   # 2. 関数の入力を取得
b.grad=C.backward(y.grad) # 3. 関数のbackwardメソッドを呼ぶ
B=b.creator # 1. 関数を取得
a=B.input   # 2. 関数の入力を取得
a.grad=B.backward(b.grad) # 3. 関数のbackwardメソッドを呼ぶ
A=a.creator # 1. 関数を取得
x=A.input   # 2. 関数の入力を取得
x.grad=A.backward(a.grad) # 3. 関数のbackwardメソッドを呼ぶ
print(x.grad)
# 逆伝播（再起的に行う場合）
y.grad=np.array(1.0)
y.backward()
print(x.grad)

1.648721270700128
3.297442541400256
3.297442541400256
